# Setup
This notebook performs the following setup actions for this example use of Amazon SageMaker Feature Store:

1. Create online-only feature groups
2. Create an Amazon Kinesis data stream
3. Create an Amazon Kinesis Data Applications (KDA) application

### Get ARN's of Lambda functions from CloudFormation stack outputs
1. InvokeFraudEndpointLambdaARN
2. StreamingAggLambdaARN

In [1]:
STACK_NAME = 'sm-fs-streaming-test'
%store STACK_NAME

Stored 'STACK_NAME' (str)


In [2]:
import sys
import boto3

cf_client = boto3.client('cloudformation')

try:
    outputs = cf_client.describe_stacks(StackName=STACK_NAME)['Stacks'][0]['Outputs']
    for o in outputs:
        if o['OutputKey'] == 'IngestLambdaFunctionARN':
            lambda_to_fs_arn = o['OutputValue']
        if o['OutputKey'] == 'PredictLambdaFunctionARN':
            lambda_to_model_arn = o['OutputValue']
        if o['OutputKey'] == 'PredictLambdaFunctionName':
            predict_lambda_name = o['OutputValue']

except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [3]:
print(f'lambda_to_model_arn: {lambda_to_model_arn}')
print(f'lambda_to_fs_arn: {lambda_to_fs_arn}')
print(f'predict_lambda_name: {predict_lambda_name}')

lambda_to_model_arn: arn:aws:lambda:us-east-1:355151823911:function:InvokeFraudEndpointLambda
lambda_to_fs_arn: arn:aws:lambda:us-east-1:355151823911:function:StreamingIngestAggFeatures
predict_lambda_name: InvokeFraudEndpointLambda


In [4]:
%store lambda_to_model_arn

Stored 'lambda_to_model_arn' (str)


In [5]:
%store predict_lambda_name

Stored 'predict_lambda_name' (str)


In [6]:
# to get the latest sagemaker python sdk
#!pip install -U sagemaker

In [7]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
#restartkernel()

### Imports and other setup

In [8]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json

role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker')
smfs_runtime = boto3.Session().client(service_name='sagemaker-featurestore-runtime')

## Create online-only feature groups
When using Amazon SageMaker Feature Store, a core design decision is the definition of feature groups. For our credit card fraud detection use case, we have decided to use two of them:

1. `cc-agg-fg` - holds aggregate features that will be updated in near real-time (streaming ingestion)
2. `cc-agg-batch-fg` - holds aggregate features that will be updated in batch

Establishing a feature group is a one-time step and is done using the `CreateFeatureGroup` API. 

Feature groups can be created as **online-only**, **offline-only**, or both **online and offline**, which replicates updates from an online store to an offline store in Amazon S3. Since our focus in this example is on demonstrating the use of the feature store for online inference and streaming aggregation of features, we make each of our feature groups online-only.

In addition to a feature group name, we provide metadata about each feature in the group. We are using a json file to define the schema, but this is not a requirement. We use a schema file to demonstrate how you might capture the feature group definitions, enabling you to recreate them consistently as you move from a development environment to a test or production environment. In our schema file, we also highlight the record identifier and the event timestamp. All feature groups must have these two features, but you get to decide how to name them.

Here is a visual summary of the feature groups we will create below.

<img src="./images/feature_groups.png" />

#### cc-agg-fg schema

In [9]:
!pygmentize schema/cc-agg-fg-schema.json

{
    "description": "Aggregated features for each credit card, batch ingestion nightly",
    "features": [
          {
              "name": "cc_num",
              "type": "bigint",
              "description": "Credit Card Number (Unique)"
          },
          {
              "name": "num_trans_last_10m",
              "type": "bigint",
              "description": "Aggregated Metric: Average number of transactions for the card aggregated by past 10 minutes"
          },
          {
              "name": "avg_amt_last_10m",
              "type": "double",
              "description": "Aggregated Metric: Average transaction amount for the card aggregated by past 10 minutes"
          },
          {
              "name": "trans_time",
              "type": "double",
              "description": "Required feature for event timestamp"
          }
      ],
    
      "record_identifier_feature_name": "cc_num",
      "event_time_feature_name": "trans_time",
      "tags": [{"Key": "Envir

#### cc-agg-batch-fg schema

In [10]:
!pygmentize schema/cc-agg-batch-fg-schema.json

{
    "description": "Aggregated features for each credit card, streamed intraday",
    
    "features": [
          {
              "name": "cc_num",
              "type": "bigint",
              "description": "Credit Card Number (Unique)"
          },
          {
              "name": "num_trans_last_1w",
              "type": "bigint",
              "description": "Aggregated Metric: Average number of transactions for the card aggregated by past 1 week"
          },
          {
              "name": "avg_amt_last_1w",
              "type": "double",
              "description": "Aggregated Metric: Average transaction amount for the card aggregated by past 1 week"
          },
          {
              "name": "trans_time",
              "type": "double",
              "description": "Required feature for event timestamp"
          }
      ],
    
      "record_identifier_feature_name": "cc_num",
      "event_time_feature_name": "trans_time",
      "tags": [{"Key": "Environment", "V

#### Utility functions to simplify creation of feature groups
`schema_to_defs` takes our schema file and returns feature definitions, and the names of the record identifier and event timestamp feature.

In [11]:
def schema_to_defs(filename):
    schema = json.loads(open(filename).read())
    
    feature_definitions = []
    
    for col in schema['Features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_definitions.append(feature)

    return feature_definitions, schema['record_identifier_feature_name'], schema['event_time_feature_name']

`schema_to_fg` creates a feature group from a schema file. If no s3 URI is passed, an online-only feature group is created.

In [12]:
from sagemaker import get_execution_role
import json

def create_feature_group_from_schema(filename, fg_name, role_arn=None, s3_uri=None):
    schema = json.loads(open(filename).read())
    
    feature_defs = []
    
    for col in schema['features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_defs.append(feature)

    record_identifier_name = schema['record_identifier_feature_name']
    event_time_name = schema['event_time_feature_name']

    if role_arn is None:
        role_arn = get_execution_role()

    if s3_uri is None:
        offline_config = {}
    else:
        offline_config = {'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': s3_uri}}}
        
    sm.create_feature_group(
        FeatureGroupName = fg_name,
        RecordIdentifierFeatureName = record_identifier_name,
        EventTimeFeatureName = event_time_name,
        FeatureDefinitions = feature_defs,
        Description = schema['description'],
        Tags = schema['tags'],
        OnlineStoreConfig = {'EnableOnlineStore': True},
        RoleArn = role_arn,
        **offline_config)

#### Create the two feature groups

In [13]:
create_feature_group_from_schema('schema/cc-agg-fg-schema.json', 'cc-agg-fg')

In [14]:
create_feature_group_from_schema('schema/cc-agg-batch-fg-schema.json', 'cc-agg-batch-fg')

#### Show that the feature store is aware of the new feature groups

In [15]:
sm.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'demo-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/demo-fg',
   'CreationTime': datetime.datetime(2020, 12, 2, 20, 25, 55, 593000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created'},
  {'FeatureGroupName': 'cc-agg-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/cc-agg-fg',
   'CreationTime': datetime.datetime(2020, 12, 9, 21, 26, 58, 284000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'},
  {'FeatureGroupName': 'cc-agg-batch-fg',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/cc-agg-batch-fg',
   'CreationTime': datetime.datetime(2020, 12, 9, 21, 27, 1, 812000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'}],
 'ResponseMetadata': {'RequestId': 'b6d845d2-3356-4b18-b14a-96cd37769a2e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b6d845d2-3356-4b18-b14a-96cd37769a2e',
   'content-type': 'application

#### Describe each feature group
Note that each feature group gets its own ARN, allowing you to manage IAM policies that control access to individual feature groups. The feature names and types are displayed, and the record identifier and event time features are called out specifically. Notice that there is only an `OnlineStoreConfig` and no `OfflineStoreConfig`, as we have decided not to replicate features offline for these groups.

In [16]:
sm.describe_feature_group(FeatureGroupName='cc-agg-fg')

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/cc-agg-fg',
 'FeatureGroupName': 'cc-agg-fg',
 'RecordIdentifierFeatureName': 'cc_num',
 'EventTimeFeatureName': 'trans_time',
 'FeatureDefinitions': [{'FeatureName': 'cc_num', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_trans_last_10m', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_amt_last_10m', 'FeatureType': 'Fractional'},
  {'FeatureName': 'trans_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2020, 12, 9, 21, 26, 58, 284000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'RoleArn': 'arn:aws:iam::355151823911:role/MySagemakerRole',
 'FeatureGroupStatus': 'Creating',
 'Description': 'Aggregated features for each credit card, batch ingestion nightly',
 'ResponseMetadata': {'RequestId': '09975786-fdc8-4913-ae66-f4563627b1cd',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '09975786-fdc8-4913-ae66-f4563627b1cd',
   'content-type': 'ap

In [17]:
sm.describe_feature_group(FeatureGroupName='cc-agg-batch-fg')

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:355151823911:feature-group/cc-agg-batch-fg',
 'FeatureGroupName': 'cc-agg-batch-fg',
 'RecordIdentifierFeatureName': 'cc_num',
 'EventTimeFeatureName': 'trans_time',
 'FeatureDefinitions': [{'FeatureName': 'cc_num', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_trans_last_1w', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_amt_last_1w', 'FeatureType': 'Fractional'},
  {'FeatureName': 'trans_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2020, 12, 9, 21, 27, 1, 812000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'RoleArn': 'arn:aws:iam::355151823911:role/MySagemakerRole',
 'FeatureGroupStatus': 'Creating',
 'Description': 'Aggregated features for each credit card, streamed intraday',
 'ResponseMetadata': {'RequestId': 'a62256f4-2d23-4947-865e-114e02ce12e2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a62256f4-2d23-4947-865e-114e02ce12e2',
   'content-type': 

## Create an Amazon Kinesis Data Stream

In [18]:
kinesis_client = boto3.client('kinesis')

In [19]:
kinesis_client.create_stream(StreamName='cc-stream', ShardCount=1)

{'ResponseMetadata': {'RequestId': 'dac10a37-1902-137b-828a-4951f1537b48',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dac10a37-1902-137b-828a-4951f1537b48',
   'x-amz-id-2': 'coRjLLudAl5dQWhsW4YASoQVXvJh42V4/9ZQpyxhl2RgQKMJm/CyzwT00wMTC8caQv/fPxw8KHEnu08ZZJie00WoUIeiaae5',
   'date': 'Wed, 9 Dec 2020 21:27:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [20]:
kinesis_client.list_streams()

{'StreamNames': ['cc-stream',
  'cc-trans-stream',
  'mie-sagemaker-Analytics-YVEC3N3LMXMF-AnalyticsStream-FTZ77HHZG1GM'],
 'HasMoreStreams': False,
 'ResponseMetadata': {'RequestId': 'ef108f4e-38dc-8f52-b75b-cc28d08de761',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ef108f4e-38dc-8f52-b75b-cc28d08de761',
   'x-amz-id-2': 'EZ+9AdF17HgaK85ZLxOe9NNHUvypTShxgnvqPnMRZUUIuDkHkWyUKSdW/CHSXFrFhtREvjeaPnBVAH2vMfaqRn26yYKmqV50',
   'date': 'Wed, 9 Dec 2020 21:27:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '138'},
  'RetryAttempts': 0}}

In [21]:
kinesis_client.describe_stream(StreamName='cc-stream')

{'StreamDescription': {'StreamName': 'cc-stream',
  'StreamARN': 'arn:aws:kinesis:us-east-1:355151823911:stream/cc-stream',
  'StreamStatus': 'CREATING',
  'Shards': [],
  'HasMoreShards': False,
  'RetentionPeriodHours': 24,
  'StreamCreationTimestamp': datetime.datetime(2020, 12, 9, 21, 27, 1, tzinfo=tzlocal()),
  'EnhancedMonitoring': [{'ShardLevelMetrics': []}],
  'EncryptionType': 'NONE'},
 'ResponseMetadata': {'RequestId': 'f752dd63-6804-a926-af19-9e058055c115',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f752dd63-6804-a926-af19-9e058055c115',
   'x-amz-id-2': 'KDgNrcr9y+SrsyMYCZEXFxEi3cFrVDvVSnQwmZvf0lGUnp/nkbmb/FNhOibAYTiC4QXk5LfV7KpAKm19QKZcgSTcyecZQtKs',
   'date': 'Wed, 9 Dec 2020 21:27:02 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '316'},
  'RetryAttempts': 0}}

In [22]:
import time
active_stream = False
while not active_stream:
    status = kinesis_client.describe_stream(StreamName='cc-stream')['StreamDescription']['StreamStatus']
    if (status == 'CREATING'):
        print('Waiting for the Kinesis stream to become active...')
        time.sleep(20)  
    elif (status == 'ACTIVE'): 
        active_stream = True
        print('ACTIVE')

Waiting for the Kinesis stream to become active...
ACTIVE


In [23]:
stream_arn = kinesis_client.describe_stream(StreamName='cc-stream')['StreamDescription']['StreamARN']

In [24]:
stream_arn

'arn:aws:kinesis:us-east-1:355151823911:stream/cc-stream'

## Map the Kinesis stream as an event source for Lambda fraud detection

In [25]:
lambda_client = boto3.client('lambda')

lambda_client.create_event_source_mapping(EventSourceArn=stream_arn,
                                          FunctionName=lambda_to_model_arn,
                                          StartingPosition='LATEST',
                                          Enabled=True,
                                          MaximumRecordAgeInSeconds=60*10
                                          ) #DestinationConfig would handle discarded records

{'ResponseMetadata': {'RequestId': '09cf7f5e-d595-476f-bb0d-6289fdd76fc8',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Wed, 09 Dec 2020 21:27:22 GMT',
   'content-type': 'application/json',
   'content-length': '711',
   'connection': 'keep-alive',
   'x-amzn-requestid': '09cf7f5e-d595-476f-bb0d-6289fdd76fc8'},
  'RetryAttempts': 0},
 'UUID': 'c8d363e6-4435-48a7-865e-98360a6d62e5',
 'BatchSize': 100,
 'MaximumBatchingWindowInSeconds': 0,
 'ParallelizationFactor': 1,
 'EventSourceArn': 'arn:aws:kinesis:us-east-1:355151823911:stream/cc-stream',
 'FunctionArn': 'arn:aws:lambda:us-east-1:355151823911:function:InvokeFraudEndpointLambda',
 'LastModified': datetime.datetime(2020, 12, 9, 21, 27, 22, 748000, tzinfo=tzlocal()),
 'LastProcessingResult': 'No records processed',
 'State': 'Creating',
 'StateTransitionReason': 'User action',
 'DestinationConfig': {'OnFailure': {}},
 'MaximumRecordAgeInSeconds': 600,
 'BisectBatchOnFunctionError': False,
 'MaximumRetryAttempts': -1}

## Create an Amazon Kinesis Data Applications (KDA) application

In [26]:
kda_client = boto3.client('kinesisanalytics')

In [27]:
sql_code = 'CREATE OR REPLACE STREAM "DESTINATION_SQL_STREAM" (\n' + \
                '"cc_num"              BIGINT,\n' + \
                '"num_trans_last_10m"  SMALLINT,\n' + \
                '"avg_amt_last_10m"    REAL\n);\n\n' + \
            'CREATE OR REPLACE PUMP "STREAM_PUMP" AS\n' + \
            'INSERT INTO "DESTINATION_SQL_STREAM"\n' + \
                'SELECT STREAM "cc_num", \n' + \
                    'COUNT(*) OVER LAST_10_MINUTES, \n' + \
                    'AVG("amount") OVER LAST_10_MINUTES\n' + \
                    'FROM "SOURCE_SQL_STREAM_001"\n' + \
                    'WINDOW LAST_10_MINUTES AS (\n' + \
                        'PARTITION BY "cc_num"\n' + \
                        'RANGE INTERVAL \'10\' MINUTE PRECEDING);\n'

In [28]:
kda_inputs = [{
                'NamePrefix': 'SOURCE_SQL_STREAM',
                'KinesisStreamsInput': {
                       'ResourceARN': stream_arn,
                       'RoleARN': role
                },
                'InputSchema': {
                      'RecordFormat': {
                          'RecordFormatType': 'JSON',
                          'MappingParameters': {
                              'JSONMappingParameters': {
                                  'RecordRowPath': '$'
                              }
                          },
                      },
                      'RecordEncoding': 'UTF-8',
                      'RecordColumns': [
                          {'Name': 'cc_num',  'Mapping': '$.cc_num',   'SqlType': 'DECIMAL(1,1)'},
                          {'Name': 'merchant','Mapping': '$.merchant', 'SqlType': 'VARCHAR(64)'},
                          {'Name': 'amount', 'Mapping': '$.amount', 'SqlType': 'REAL'},
                          {'Name': 'zip_code', 'Mapping': '$.zip_code', 'SqlType': 'INTEGER'}
                      ]
                }
              }                         
             ]

<h3> Create Kinesis Data Analytics Application </h3>

We first lookup Lambda ARNs from CloudFormation output, then create a Kinesis Data Analytics application that connects its output to the Streaming Lambda. This Lambda will ingest the records and write them to the SageMaker Feature Group.

In [29]:
kda_outputs = [{'LambdaOutput': {'ResourceARN': lambda_to_fs_arn, 'RoleARN': role},
                'Name': 'DESTINATION_SQL_STREAM',
                'DestinationSchema': {'RecordFormatType': 'JSON'}}]

In [30]:
kda_outputs

[{'LambdaOutput': {'ResourceARN': 'arn:aws:lambda:us-east-1:355151823911:function:StreamingIngestAggFeatures',
   'RoleARN': 'arn:aws:iam::355151823911:role/MySagemakerRole'},
  'Name': 'DESTINATION_SQL_STREAM',
  'DestinationSchema': {'RecordFormatType': 'JSON'}}]

In [31]:
kda_client.create_application(ApplicationName='cc-agg-app', 
                              Inputs=kda_inputs,
                              Outputs=kda_outputs,
                              ApplicationCode=sql_code)

{'ApplicationSummary': {'ApplicationName': 'cc-agg-app',
  'ApplicationARN': 'arn:aws:kinesisanalytics:us-east-1:355151823911:application/cc-agg-app',
  'ApplicationStatus': 'READY'},
 'ResponseMetadata': {'RequestId': 'd8b9322a-7e90-4c61-beb1-c42c6e45a6a6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd8b9322a-7e90-4c61-beb1-c42c6e45a6a6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '173',
   'date': 'Wed, 09 Dec 2020 21:27:23 GMT'},
  'RetryAttempts': 0}}

In [32]:
kda_client.describe_application(ApplicationName='cc-agg-app')

{'ApplicationDetail': {'ApplicationName': 'cc-agg-app',
  'ApplicationARN': 'arn:aws:kinesisanalytics:us-east-1:355151823911:application/cc-agg-app',
  'ApplicationStatus': 'READY',
  'CreateTimestamp': datetime.datetime(2020, 12, 9, 21, 27, 23, tzinfo=tzlocal()),
  'LastUpdateTimestamp': datetime.datetime(2020, 12, 9, 21, 27, 23, tzinfo=tzlocal()),
  'InputDescriptions': [{'InputId': '1.1',
    'NamePrefix': 'SOURCE_SQL_STREAM',
    'InAppStreamNames': ['SOURCE_SQL_STREAM_001'],
    'KinesisStreamsInputDescription': {'ResourceARN': 'arn:aws:kinesis:us-east-1:355151823911:stream/cc-stream',
     'RoleARN': 'arn:aws:iam::355151823911:role/MySagemakerRole'},
    'InputSchema': {'RecordFormat': {'RecordFormatType': 'JSON',
      'MappingParameters': {'JSONMappingParameters': {'RecordRowPath': '$'}}},
     'RecordEncoding': 'UTF-8',
     'RecordColumns': [{'Name': 'cc_num',
       'Mapping': '$.cc_num',
       'SqlType': 'DECIMAL(1,1)'},
      {'Name': 'merchant', 'Mapping': '$.merchant', 

In [33]:
kda_client.start_application(ApplicationName='cc-agg-app',
                             InputConfigurations=[{'Id': '1.1',
                                                   'InputStartingPositionConfiguration': 
                                                     {'InputStartingPosition':'NOW'}}])

{'ResponseMetadata': {'RequestId': '80db3937-4f1d-49f2-bed1-516efdfb5db5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '80db3937-4f1d-49f2-bed1-516efdfb5db5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Wed, 09 Dec 2020 21:27:23 GMT'},
  'RetryAttempts': 0}}

# NOTES before finalizing
This nb needs both KinesisFullAccess and KinesisAnalyticsFullAccess

Role passed to create KDA app needs Lambda execution

nb role needs trust relationship with kinesisanalytics.amazonaws.com for passing role

In [34]:
print(f'boto3: {boto3.__version__}')


boto3: 1.16.19
